In [1]:
# still need to drop outliers and build normal dist

# Average Daily Price Volatility

# Question: on what days does QQQ consistently show the higher than usual high-low spreads?

In [2]:
import pandas as pd
import numpy as np
import matplotlib as mpl 
import matplotlib.pyplot as plt 

In [3]:
# get daily historical market data from Yahoo for QQQ
data = pd.read_csv('QQQ.csv')

In [4]:
### spread_grid ###

# get unique days and years from data
unique_days = []
unique_years = []
for i, row in data.iterrows():
    date = row['Date']
    day = date[:5]
    year = date[6:]
    # in python 3 --> import collections; use OrderedSet
    unique_days = list(set(unique_days + [day]))
    unique_years = list(set(unique_years + [year]))
        
# create a df with index containing unique years and cols for each unique day
spread_grid = pd.DataFrame(index = unique_years, columns = unique_days)

for i, row in data.iterrows():
    date = row['Date']
    day = date[:5]
    year = date[6:]
    spread = row['High'] - row['Low']
    spread_grid.at[year, day] = spread

In [5]:
### summary statistics for days in spread_grid ###
summary = pd.DataFrame(columns = spread_grid.columns.values)
summary.loc['count'] = spread_grid.count()
summary.loc['std'] = spread_grid.std()
summary.loc['min'] = spread_grid.min()
summary.loc['mean'] = spread_grid.mean()
summary.loc['median'] = spread_grid.median()
summary.loc['max'] = spread_grid.max()

summary.loc['Q1'] = None
summary.loc['Q3'] = None
summary.loc['outlier range'] = None
summary.loc['outlier count'] = None
for col in summary:
    summary.at['Q1', col] = spread_grid[col].quantile(.25)
    summary.at['Q3', col] = spread_grid[col].quantile(.75)
    
    # outlier range
    q1 = summary[col]['Q1']
    q3 = summary[col]['Q3']
    iqr = q3 - q1
    outlier_range = [q1 - 1.5 * iqr, q3 + 1.5 * iqr]
    summary.at['outlier range', col] = outlier_range
    
    # outlier count
    spreads = spread_grid[col].values
    n_outliers = 0
    for spread in spreads:
        if spread < outlier_range[0] or spread > outlier_range[1]:
            n_outliers += 1
    summary.at['outlier count', col] = n_outliers 
    
summary = summary.reindex(['count', 'outlier range', 'outlier count', 'std', 'min',  
                           'Q1','mean', 'median', 'Q3', 'max'])

summary

,07/18,07/19,07/10,07/11,07/12,07/13,07/14,07/15,07/16,07/17,...,05/16,05/17,05/14,05/15,05/12,05/13,05/10,05/11,05/18,05/19
count,16,16,15,16,16,15,14,14,15,15,...,16,16,15,15,14,14,16,15,15,14
outlier range,"[-0.368756125, 2.261256875]","[-0.26768725, 1.87948475]","[-0.40750575, 2.57251225]","[-0.300000625, 2.259998375]","[-0.27155925, 1.94593075]","[0.10687925, 1.55186925]","[-0.333757375, 1.976255625]","[0.267297625, 1.087622625]","[-0.22500525, 2.09500475]","[-0.93749475, 3.12249925]",...,"[-0.58626475, 2.64375925]","[-1.68770125, 4.37950275]","[-0.13499425, 1.94499175]","[-0.95500575, 3.16500425]","[-0.18500475, 2.35500325]","[-0.35248775, 2.02748825]","[-1.00876475, 2.98127325]","[0.4500005, 1.5699965]","[-0.25000275, 2.27000325]","[0.09625225, 1.78624425]"
outlier count,1,2,0,1,2,1,1,3,0,1,...,3,0,2,2,2,1,2,1,1,1
std,0.607261,0.886053,0.55084,0.628708,0.745103,0.572876,0.532124,0.460366,0.457905,0.830128,...,1.06672,0.872804,0.612325,1.56425,0.885886,0.747361,1.56614,0.913326,0.878523,0.805902
min,0.2,0.349999,0.470002,0.360001,0.449997,0.260002,0.360001,0.349998,0.320004,0.380001,...,0.41,0.279999,0.600002,0.420001,0.400009,0.449999,0.34,0.460003,0.300004,0.319999
Q1,0.617499,0.537502,0.710001,0.659999,0.56,0.64875,0.532497,0.57492,0.644998,0.585003,...,0.624994,0.5875,0.645001,0.589998,0.767498,0.540003,0.4875,0.869999,0.694999,0.729999
mean,1.07199,1.04115,1.17567,1.05922,1.03891,0.904833,0.881965,0.832834,0.959229,1.19583,...,1.30422,1.40545,1.044,1.51933,1.22991,0.997322,1.42602,1.179,1.18462,1.08955
median,0.97,0.725002,1.04,0.955,0.814998,0.75,0.640002,0.744999,0.889999,0.939994,...,0.899998,1.33,0.780001,1.01,0.974999,0.685002,0.694997,1,1.04999,0.974998
Q3,1.275,1.0743,1.45501,1.3,1.11437,1.01,1.11,0.780001,1.225,1.6,...,1.4325,2.1043,1.165,1.62,1.4025,1.135,1.48501,1.15,1.325,1.1525
max,2.42188,3.28125,2.48,2.9375,3.125,2.625,2.3125,1.89,1.96,3.36999,...,3.46,2.75,2.77,6,3.5625,3.2,5.48438,4.375,3.98438,3.6875


In [6]:
### spread_grid with outliers removed ###

no_outliers = spread_grid
for col in no_outliers:
    floor = summary[col]['outlier range'][0]
    ciel = summary[col]['outlier range'][1]
    no_outliers[col] = no_outliers.loc[no_outliers[col] < ciel]
    no_outliers[col] = no_outliers.loc[no_outliers[col] > floor]
    #no_outliers[no_outliers[col] > ciel].dropna()
    #no_outliers.loc[no_outliers[col] < floor, col] = None
    #no_outliers.loc[no_outliers[col] > ciel, col] = None
no_outliers

,07/18,07/19,07/10,07/11,07/12,07/13,07/14,07/15,07/16,07/17,...,05/16,05/17,05/14,05/15,05/12,05/13,05/10,05/11,05/18,05/19
02,1.05,1.05,1.05,1.05,1.05,NaN,NaN,NaN,1.05,1.05,...,1.05,1.05,1.05,1.05,NaN,1.05,1.05,NaN,NaN,NaN
03,0.629999,NaN,0.629999,0.629999,NaN,NaN,0.629999,0.629999,0.629999,0.629999,...,0.629999,NaN,0.629999,0.629999,0.629999,0.629999,NaN,NaN,NaN,0.629999
00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01,1.71,1.71,1.71,1.71,1.71,1.71,NaN,NaN,1.71,1.71,...,NaN,1.71,NaN,1.71,NaN,NaN,1.71,1.71,1.71,NaN
06,0.790001,0.790001,0.790001,0.790001,0.790001,0.790001,0.790001,NaN,NaN,0.790001,...,0.790001,0.790001,NaN,0.790001,0.790001,NaN,0.790001,0.790001,0.790001,0.790001
07,0.529999,0.529999,0.529999,0.529999,0.529999,0.529999,NaN,NaN,0.529999,0.529999,...,0.529999,0.529999,0.529999,0.529999,NaN,NaN,0.529999,0.529999,0.529999,NaN
04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
05,0.2,0.2,NaN,0.2,0.2,0.2,0.2,NaN,NaN,NaN,...,0.2,0.2,NaN,NaN,0.2,0.2,0.2,NaN,0.2,0.2
19,2.09999,NaN,2.09999,2.09999,2.09999,NaN,NaN,2.09999,2.09999,2.09999,...,NaN,2.09999,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,1.06,1.06,1.06,1.06,NaN,1.06,NaN,NaN,1.06,NaN,...,1.06,1.06,1.06,1.06,NaN,NaN,1.06,1.06,1.06,NaN


In [12]:
no_outliers.loc['count'] = no_outliers.count()
no_outliers.loc['count']

07/18    16
07/19    12
07/10    13
07/11    16
07/12    12
07/13    10
07/14     9
07/15     6
07/16    11
07/17    12
03/18    10
03/19    10
03/14    14
03/15    13
03/16    10
03/17     9
03/10     8
03/11     9
03/12    10
03/13    12
01/18    12
01/19    10
01/12    10
01/13     8
01/10    13
01/11    12
01/16    10
01/17    13
01/14     9
01/15     8
         ..
10/23    12
10/22    11
04/15     9
04/14     7
04/17    11
04/16    10
04/11    16
10/28     9
04/13     8
04/12    12
02/19     8
02/18     5
02/13    12
02/12    10
02/11     9
02/10     9
02/17     6
02/16    10
02/15    12
02/14    13
05/16    14
05/17    13
05/14     9
05/15    12
05/12     9
05/13     9
05/10    12
05/11     9
05/18    10
05/19     9
Name: count, Length: 363, dtype: object

In [13]:
summary.loc['count']

07/18    16
07/19    16
07/10    15
07/11    16
07/12    16
07/13    15
07/14    14
07/15    14
07/16    15
07/17    15
03/18    14
03/19    15
03/14    16
03/15    16
03/16    15
03/17    14
03/10    14
03/11    14
03/12    15
03/13    15
01/18    13
01/19    11
01/12    14
01/13    13
01/10    15
01/11    15
01/16    12
01/17    12
01/14    14
01/15    11
         ..
10/23    15
10/22    15
04/15    14
04/14    12
04/17    15
04/16    15
04/11    16
10/28    14
04/13    14
04/12    16
02/19    11
02/18    10
02/13    15
02/12    14
02/11    14
02/10    13
02/17    11
02/16    11
02/15    13
02/14    15
05/16    16
05/17    16
05/14    15
05/15    15
05/12    14
05/13    14
05/10    16
05/11    15
05/18    15
05/19    14
Name: count, Length: 363, dtype: object

In [ ]:
spread_grid

In [ ]:
summary

In [ ]:
### volatility_score model ###

# volatility_score is determined by the sum of mean and median for days with at least
#   min_data_pts and at most max_outliers
# the idea here is to find days that consistently show high price volitility. so, we 
#   want to limit the variability in the spreads for our top days.

summary.loc['volatility score'] = None
max_outliers = 0
min_data_pts = 10 - max_outliers
for col in summary:
    if summary[col]['count'] >= min_data_pts and summary[col]['outlier count'] <= max_outliers:
        mean = summary[col]['mean']
        median = summary[col]['median']
        std = summary[col]['std']
        maxi = summary[col]['max']
        mini = summary[col]['min']
        q1 = summary[col]['Q1']
        q3 = summary[col]['Q3']
        n_outliers = summary[col]['outlier count']
        summary[col]['volatility score'] = mean + median

In [ ]:
### top 5 days by volatility_score ###

summary = summary.transpose()
summary = summary.sort_values(by='volatility score', ascending=False)
top_5 = summary[:5]
top_5_days = top_5.index.values
top_5_spreads = {}
for day in spread_grid:
    if day in top_5_days:
        top_5_spreads[day] = spread_grid[day].values

In [ ]:
### boxplots for top 5 days ###

no_nan = []
for arr in top_5_spreads.values():
    clean_arr = np.array([])
    for val in arr:
        if not pd.isna(val):
            clean_arr = np.append(clean_arr, val)
    no_nan += [clean_arr]

# Create a figure instance
fig = plt.figure(figsize=(9, 6))

# Create an axes instance
ax = fig.add_subplot(111)

# Create the boxplot
bp = ax.boxplot(no_nan)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(spread_grid)

In [ ]:
### the datasets ###

data
spread_grid
summary
no_outliers